Başlamadan önce ileride sık sık duyacağınız ve CNN adının C harfinin temsil ettiği
konvolüsyon  ne olduğundan bahsetmek isterim. Neural Nwtwork yapısında konvolüsyon,
iki bilgi kümesinin birleştirilmesini sağlayan matematiksel bir işlemdir. CNN durumunda,
bilgiyi filtrelemek ve bir özellik haritası üretmek için giriş verilerine konvolüsyon
uygulanır. Bu filtreye çekirdek veya özellik dedektörü de denir ve boyutları örneğin
3x3 olabilir. Konvolüsyon gerçekleştirmek için çekirdek, giriş görüntüsünün üzerinden
geçerek eleman eleman matris çarpımı yapar. Her alıcı alan (konvolüsyonun gerçekleştiği
alan) için sonuç özellik haritasına yazılır. Özellik haritası tamamlanana kadar filtreyi
kaydırmaya devam ederiz. Bir diğer değiş ile resimleri ço boyutlu  matrislerden nöral ağda
işleleme girecek şekilde yeniden şekillendirilmesi, hazırlanmasıdır.

Bu not defterinde nöral ağ yapısında kullanılan ağırlık (weights) ve aktivasyon fonksiyonlarınn
çalışma mantığına bakacak ve OpenCV ile ilk temel nöral ağ protoipimizi yapacağız.

Başlarken her zaman olduğu gibi kütüphanelerimizi içe aktarıyor ve resimi üzerinde işlem
yapılabilir hale getiren fonksiyonumuzu çağırıyor, sonrasında da resimlermizi içe aktarıyoruz.

In [1]:
import cv2 as cv
import numpy as np

In [2]:
def prepare_the_image(imgPath):
    img = cv.imread(imgPath, cv.IMREAD_GRAYSCALE)
    img = img/255
    flatten = img.flatten()
    return flatten

In [3]:
flattenV = prepare_the_image("../dataset/train/vertical.png")

print("Flatten for of vertical image is, ",flattenV)

Flatten for of vertical image is,  [0. 1. 0. 0. 1. 0. 0. 1. 0.]


Şimdi ağırlıklarımızı (weights) oluşturalım. Ağırlık kavramı aslında geçtiğimiz not
defterlerinde oluşturduğumuz filtreden farklıdır. Ağırlıklar iki nöral ağ katmanı
arasında bulunan filtrelerden ibarettirler. Nöral ağ eğitilirken bu filtreleri, bundan
sonra ağırlık olarak kullanacağım, eğitim objelerini birbirlerinden ayıracak şekilde yeniden
ayarlanır tıpkı geçtiğimiz not defterinde while döngüsü ile yaptığımız gibi.

Tabi ki sistemi oldukça basite indirgedik ama temel çalışma prensibi bu şekilde. Şimdi
Numpy aracılığı ile bir ağırlık dizisi oluşturalım ve resimimiz ile işleme sokrak işleme
devam edelim.

In [4]:
weights = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [5]:
convolution = sum(flattenV * weights)

print("Convolution value is :: ", convolution)

Convolution value is ::  1.5


Yukarıda ki hücreyi çalıştırdığınız zaman çıkan sonuç görebileceğiniz gibi 1.5 oldu.
Biz ilk not defterinden beri resimlerde bulunan piksel değerlerini 0 ile 255 arasından
0 ile 1 arasına sıkıştırarak daha kolay işlem yapılabilir hale getiriyoruz. Burda da
yapmamız gereken olarak tam olarak bu. 1.5 çok fark etmesede katman sayısıa arttıkaç ve
ağırlıklar daha büyük değerler aldıkça bu 1.5 değerei binler belki daha da yüksek bir
değere ulaşabilir. Bu yüzden bu 1.5 değerini 0 ile 1 arasına sokmalıyız ama bunu belirli
bir fonksiyon doğrultusunda yapmalıyız ki elde ettiğimiz tüm değerler eşit işlensin.

Bu tarz değerleri 0 ile 1 arasına sıkıştıran fonksiyonlara aktivasyon fonksyionları denir
Biz SIGMOID aktivasyon fonksiyonu ile işlem yapacağız. aşağıda ki hüzcrede SIGMOID
fonksiyonunun python diline aktarılmış hali bulunmaktadır.

In [6]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

Şimdi daha önce elde ettiğimiz konvolüsyon değreini sigmoid fonksiyonuna sokarak
elde edeceğimiz sonucu gözlemleyelim.

In [7]:
result = sigmoid(convolution)

print("Result value is {}. (It is compressed value of {} with sigmoid function)".format(result, convolution))

Result value is 0.8175744761936437. (It is compressed value of 1.5 with sigmoid function)


Aktivasyon fonksiyonu sonucunda 1.5 değeri 0.8 değerine sıkıştırıldı. Bundan sonra basitçe
bir sınıflandırma yapabilriz. Bu sınıdlnadırma içi bir eşik değere ihtiyacımız olacaktır.
Eğer hedef resimi nöral ağdan geçirdikten sonra elde ettiğimiz değer eşik değerin
altında ise A sınıfı eşik değerin üstünde ise B sınıfına ait şeklinde. Bizim durumumuzda
eşik değerimiz 0.5 olacaktır. Eğer elde edilen sonuç 0.5 değerinde büyük ise dikey,
küçük ise yatay olacaktır.

In [8]:
if result < 0.5:
    print("Image is VERTICAL")
elif result > 0.5:
    print("Image is HORIZONTAL")

Image is HORIZONTAL


Şuana kadar yazdığımız bütün kodu pythın scripti olarak bulabilirsiniz. Oluşturduğumuz
nöral ağı kullanabilir bir  fonksiyona dönüştürdüm. Ama şuan nöral ağımızın ağırlıkları
henüz uygun şekilde ayarlanmadığı için dikey ve yatay resim arasındaki farkı ayırt edmiyor.

Bir sonraki not defterinde ağırlıkları dikey resim için 0.5 değerinin üstünde yatay resim için
ise 0.5 değerinin altında olacak şekilde ayarlanmasını sağlayacağız.